In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ddm import Fittable, Model, Sample, Bound
from ddm.models import LossRobustBIC, DriftConstant, NoiseConstant, BoundConstant, OverlayNonDecision, Drift
from ddm.functions import fit_adjust_model, display_model
import ddm.plot

In [2]:
# import my file
my_file = pd.read_csv('../2_cleaned/df_verbose.csv', sep = ',')
# reduce to one subject
my_file = my_file[my_file.subject == 'AD']
# only get 'go responses'
my_file = my_file[my_file.goResp == 1]
# compute an answer variable
my_file['answer'] = 1-abs(my_file.goResp - my_file.hitGoal) 
# compute the reaction time in ms
my_file.loc[:,'handmoveTimeMsGo'] = my_file['handmoveTimeMsGo']/1000 
# drift diffusion models track the evidence for one output over another.
# in this implementation, the upper bound represents a correct outcome, the lower bound represents an incorrect outcome
# therefore, we need to scale our evidence (probability of a hit) such that it reflects the similarity to the true outcome 
# of the trial. (Large, when there is no difference between the true outcome and the evidence), 0 when it's neutral, 1 when 
# there's evidence for the opposite outcome

my_file.loc[:,'sampleProbHit_01'] = 2 * (0.5 - abs(my_file['hitGoal']-my_file['sampleProbHit_01']))
my_file.loc[:,'sampleProbHit_02'] = 2 * (0.5 - abs(my_file['hitGoal']-my_file['sampleProbHit_02'])) 
my_file.loc[:,'sampleProbHit_03'] = 2 * (0.5 - abs(my_file['hitGoal']-my_file['sampleProbHit_03'])) 
my_file.loc[:,'sampleProbHit_04'] = 2 * (0.5 - abs(my_file['hitGoal']-my_file['sampleProbHit_04'])) 
my_file.loc[:,'sampleProbHit_05'] = 2 * (0.5 - abs(my_file['hitGoal']-my_file['sampleProbHit_05'])) 
my_file.loc[:,'sampleProbHit_06'] = 2 * (0.5 - abs(my_file['hitGoal']-my_file['sampleProbHit_06'])) 

# reduce my data file to the neccesary columns
my_file = my_file.loc[:,['handmoveTimeMsGo', 'answer', 'sampleProbHit_01', 'sampleProbHit_02', 'sampleProbHit_03', 'sampleProbHit_04', 'sampleProbHit_05', 'sampleProbHit_06']]

# drop all rows that contain nans and reset the index 
my_file.dropna(axis = 1, inplace = True)
my_file.reset_index(drop = True, inplace = True)

# turn my datafile into a pyDDM sample
sample = Sample.from_pandas_dataframe(my_file, rt_column_name="handmoveTimeMsGo", correct_column_name="answer")

In [3]:
class BoundCollapsingExponentialDelay(Bound):
    """Bound collapses exponentially over time.

    Takes three parameters: 

    `B` - the bound at time t = 0.
    `tau` - the time constant for the collapse, should be greater than
    zero.
    `t1` - the time at which the collapse begins, in seconds
    """
    name = "Delayed exponential collapsing bound"
    required_parameters = ["B", "tau", "t1"]
    def get_bound(self, t, conditions, **kwargs):
        if t <= self.t1:
            return self.B
        if t > self.t1:
            return self.B * np.exp(-self.tau*(t-self.t1))

In [4]:
# Define the Drift for the Third Model
class ContinuousUpdate(Drift):
    name = "continuously updating drifts"
    required_conditions = ["sampleProbHit_01", "sampleProbHit_02", "sampleProbHit_03", "sampleProbHit_04", "sampleProbHit_05", "sampleProbHit_06"]
    required_parameters = ["scale"]
    time_schema = np.linspace(0,1,6)
    def get_drift(self, t, conditions, **kwargs):
        passed = self.time_schema[(self.time_schema - t)<=0]
        prob = self.required_conditions[np.argmax(passed)]
        return conditions[prob]*self.scale

In [5]:
## define the third model
model3_fit = Model(name='drift changes with every new sample',
                  drift= ContinuousUpdate(scale = Fittable(minval=1, maxval=10)),
                  noise=NoiseConstant(noise=Fittable(minval=.5, maxval=4)),
                  bound=BoundCollapsingExponentialDelay(B=1,
                                           tau=Fittable(minval=0.1, maxval=5),
                                           t1=Fittable(minval=0, maxval=1)),
                  overlay=OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                  dx=.001, dt=.01, T_dur=1)

fit_adjust_model(sample, model3_fit,lossfunction=LossRobustBIC, verbose=False)

Params [3.08707002 0.95726297 5.         0.14178993 0.24401554] gave -1764.0105370197462


Model(name='drift changes with every new sample', drift=ContinuousUpdate(scale=Fitted(3.0870700180883444, minval=1, maxval=10)), noise=NoiseConstant(noise=Fitted(0.9572629706934258, minval=0.5, maxval=4)), bound=BoundCollapsingExponentialDelay(B=1, tau=Fitted(5.0, minval=0.1, maxval=5), t1=Fitted(0.14178993487441863, minval=0, maxval=1)), IC=ICPointSourceCenter(), overlay=OverlayNonDecision(nondectime=Fitted(0.2440155395803223, minval=0, maxval=1)), dx=0.001, dt=0.01, T_dur=1, fitresult=FitResult(fitting_method='differential_evolution', method='auto', loss='BIC', value=-1764.0105370197462, nparams=5, samplesize=1296, mess=''))

In [6]:
import pickle
path_models = '../2-3_Fitted/pyDDM_Models/'

with open(path_models + 'ddm3_30_04.pkl', 'wb') as output:
    pickle.dump(model3_fit, output, pickle.HIGHEST_PROTOCOL)